In [10]:
import numpy as np
import scipy
import utils
from utils import HouseHolder, QR, SVD

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Problem 1: SVD by Two-Phase Approach

## Phase-I: Golub-Kahan Bidiagonalization

In [11]:
%precision 8

'%.8f'

In [12]:
A = np.array([[0, 0, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 0],
              [2, 5, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 0, 0]], dtype=np.float64)


# A = np.array([[1, 0, 1],
#               [2, 5**.5, 0],
#               [0, 0, 1],
#               [0, 0, 1]])
B, Qt, P = SVD.svd_phaseI(A)
print(B)


[[ 2. -5.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


In [13]:
print(Qt @ A @ P)

[[ 2. -5.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


Todo: We may not need to form the reflection matrix H explicitly during Golub-Kahan bidiagonalization.

## Phase-II-A

**Test SVD**

In [14]:
%precision 4

'%.4f'

In [52]:
m=  n = 500
A = np.random.rand(m,n)


In [59]:
U, S, Vt = SVD.svd(A)
# print(U@np.diag(S)@Vt)
print(np.abs(U@np.diag(S)@Vt - A)< 1e-8)
U, S, Vt

phaseI: 2.7893500328063965
phaseII: 7.983047008514404
[[ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]


(array([[-0.0442,  0.0123, -0.0125, ..., -0.0313, -0.0161,  0.0426],
        [-0.0423,  0.0331,  0.0902, ...,  0.0054, -0.0952, -0.0305],
        [-0.0452,  0.0379,  0.0599, ...,  0.0839,  0.0018, -0.0327],
        ...,
        [-0.044 ,  0.019 , -0.0417, ..., -0.0413,  0.0358, -0.0138],
        [-0.0428,  0.0335,  0.0784, ..., -0.0137,  0.0205,  0.0247],
        [-0.0427, -0.0421,  0.0407, ...,  0.0018, -0.0466,  0.0153]]),
 array([2.5005e+02, 1.2890e+01, 1.2762e+01, 1.2615e+01, 1.2502e+01,
        1.2464e+01, 1.2428e+01, 1.2290e+01, 1.2255e+01, 1.2196e+01,
        1.2140e+01, 1.2073e+01, 1.1935e+01, 1.1934e+01, 1.1905e+01,
        1.1852e+01, 1.1791e+01, 1.1745e+01, 1.1722e+01, 1.1671e+01,
        1.1619e+01, 1.1585e+01, 1.1549e+01, 1.1459e+01, 1.1413e+01,
        1.1377e+01, 1.1354e+01, 1.1320e+01, 1.1284e+01, 1.1184e+01,
        1.1155e+01, 1.1108e+01, 1.1086e+01, 1.1056e+01, 1.1028e+01,
        1.0992e+01, 1.0929e+01, 1.0871e+01, 1.0889e+01, 1.0831e+01,
        1.0770e+01, 1.0756e

In [34]:
U, S, Vt  = scipy.linalg.svd(A, full_matrices=False)
print(U@np.diag(S)@Vt)
U, S, Vt

[[0.2421 0.1893 0.3337 ... 0.251  0.6522 0.0602]
 [0.8174 0.4575 0.1309 ... 0.0714 0.913  0.9533]
 [0.1001 0.0747 0.868  ... 0.5586 0.3942 0.9228]
 ...
 [0.1837 0.5565 0.1956 ... 0.4669 0.5691 0.2415]
 [0.3115 0.7031 0.0776 ... 0.3628 0.9256 0.2496]
 [0.086  0.386  0.8097 ... 0.9425 0.4488 0.7326]]


(array([[-0.0496, -0.0268,  0.0749, ..., -0.0228, -0.044 ,  0.0758],
        [-0.0479,  0.0754,  0.0608, ..., -0.0283, -0.0524,  0.0016],
        [-0.0518,  0.0321,  0.0225, ...,  0.0406,  0.039 ,  0.0245],
        ...,
        [-0.0492,  0.0389, -0.0504, ...,  0.0252, -0.0258, -0.0712],
        [-0.0495, -0.0047, -0.0201, ..., -0.0316,  0.054 ,  0.0026],
        [-0.0484, -0.1018, -0.0299, ...,  0.0256, -0.0542,  0.1023]]),
 array([2.0018e+02, 1.1455e+01, 1.1220e+01, 1.1159e+01, 1.1129e+01,
        1.1043e+01, 1.0987e+01, 1.0778e+01, 1.0694e+01, 1.0666e+01,
        1.0645e+01, 1.0595e+01, 1.0546e+01, 1.0494e+01, 1.0479e+01,
        1.0368e+01, 1.0302e+01, 1.0285e+01, 1.0229e+01, 1.0208e+01,
        1.0172e+01, 1.0104e+01, 1.0072e+01, 1.0043e+01, 9.9971e+00,
        9.9519e+00, 9.9203e+00, 9.9040e+00, 9.8397e+00, 9.7968e+00,
        9.7340e+00, 9.6832e+00, 9.6338e+00, 9.5993e+00, 9.5644e+00,
        9.5173e+00, 9.4658e+00, 9.4003e+00, 9.3936e+00, 9.3374e+00,
        9.3015e+00, 9.2925e